# Train Model with DPO - Gemma3-1B-it

In [1]:
!pip install -q  datasets trl accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
from huggingface_hub import login
hf_token = input("Please enter your Hugging Face API Key:")
login(token=hf_token)

In [4]:
dataset = load_dataset("Eshita-ds/cot-llm-dpo-dataset-final")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/649 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/158k [00:00<?, ?B/s]

valid-00000-of-00001.parquet:   0%|          | 0.00/38.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/270 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/30 [00:00<?, ? examples/s]

In [5]:
model_name = "google/gemma-3-1b-it"

# model = AutoModelForCausalLM.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.gradient_checkpointing_enable()
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token # set pad token

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [6]:
def format_chat_prompt(user_input, system_message="You are a helpful AI Tutor."):

    # Format user message
    user_prompt = f"<|chat_start|>user\n{user_input}<|chat_end|>\n"

    # Start assistant's turn
    tutor_prompt = "<|chat_start|>tutor\n"

    # Combine prompts
    formatted_prompt = user_prompt + tutor_prompt

    return formatted_prompt

In [7]:
# Set up text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Example prompt
prompt = format_chat_prompt(dataset['valid']['prompt'][0][0]['content'])

# Generate output
outputs = generator(prompt, max_length=700, truncation=True, num_return_sequences=1, temperature=0.7)

print(outputs[0]['generated_text'])

Device set to use cuda:0


<|chat_start|>user
You are an AI tutor that thinks and provides detailed and step-by-step explanations for the provided maths question.

**Question**: a circle of radius 1 is internally tangent to two circles of radius 2 at points  $a$ and $b$, where $ab$ is a diameter of the smaller circle. what is the  area of the region, shaded in the figure, that is outside  the smaller circle and inside each of the two larger circles? express your answer in terms of $\\pi$ and in simplest radical form.\n\n[asy]\nunitsize(1cm);\npair a = (0,-1), b = (0,1);\nfill(arc(a,2,30,90)--arc((0,0),1,90,-90)--arc(b,2,270,330)--cycle,gray(0.7));\nfill(arc(a,2,90,150)--arc(b,2,210,270)--arc((0,0),1,270,90)--cycle,gray(0.7));\ndraw(circle((0,-1),2));\ndraw(circle((0,1),2));\ndraw(circle((0,0),1));\ndraw((0,0)--(0.71,0.71),arrow);\ndraw((0,-1)--(-1.41,-2.41),arrow);\ndraw((0,1)--(1.41,2.41),arrow);\ndot((0,-1));\ndot((0,1));\nlabel("$a$",a,s);\nlabel("$b$",b,n);\nlabel("2",(0.7,1.7),n);\nlabel("2",(-0.7,-1.7),n);

In [8]:
ft_model_name = model_name.split('/')[1].replace("Instruct", "DPO")

training_args = DPOConfig(
    output_dir=ft_model_name,
    logging_steps=25,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="epoch",
    eval_strategy="epoch",
    eval_steps=1,
)

device = torch.device('cuda')

In [9]:
trainer = DPOTrainer(
    model=model,
    args=training_args,
    processing_class=tokenizer,
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'],
)
trainer.train()

Extracting prompt in train dataset:   0%|          | 0/270 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/270 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/270 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/30 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/30 [00:00<?, ? examples/s]

...

It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
0,0.650600,0.567839,0.711198,0.390495,0.700000,0.320915,-542.266663,-482.799988,-6.718750,-6.820833


Epoch,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
0,0.650600,0.567839,0.711198,0.390495,0.700000,0.320915,-542.266663,-482.799988,-6.718750,-6.820833
1,0.565200,0.548568,0.923698,0.560742,0.733333,0.364030,-539.066650,-481.333344,-6.706250,-6.808333
2,0.514100,0.502865,0.926953,0.444124,0.800000,0.484180,-539.599976,-482.533325,-6.706250,-6.810417


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=99, training_loss=0.5572102864583334, metrics={'train_runtime': 509.8745, 'train_samples_per_second': 1.589, 'train_steps_per_second': 0.194, 'total_flos': 0.0, 'train_loss': 0.5572102864583334, 'epoch': 2.977777777777778})

In [10]:
# Load the fine-tuned model
ft_model = trainer.model

In [11]:
model_id = f"Eshita-ds/{ft_model_name}"
trainer.push_to_hub(model_id)

events.out.tfevents.1743822753.b371e91ce015.745.0:   0%|          | 0.00/10.8k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/6.26k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Eshita-ds/gemma-3-1b-it/commit/dc88fcfe6f91cbaeca1075d5f7bf8fce7e8252ed', commit_message='Eshita-ds/gemma-3-1b-it', commit_description='', oid='dc88fcfe6f91cbaeca1075d5f7bf8fce7e8252ed', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Eshita-ds/gemma-3-1b-it', endpoint='https://huggingface.co', repo_type='model', repo_id='Eshita-ds/gemma-3-1b-it'), pr_revision=None, pr_num=None)

In [ ]:
# Set up text generation pipeline
generator = pipeline("text-generation", model=ft_model, tokenizer=tokenizer)

# Example prompt
prompt = format_chat_prompt(dataset['valid']['prompt'][0][0]['content'])

# Generate output
outputs = generator(prompt, max_length=700, truncation=True, num_return_sequences=1, temperature=0.7)

print(outputs[0]['generated_text'])